In [ ]:
%%capture
!pip install ffmpeg
!pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]
!pip install sounddevice
!pip install transformers -U
!pip install ffpyplayer
!apt-get install libportaudio2
!pip install ffmpeg-python
!pip install huggingface_hub
!pip install -q torch transformers accelerate bitsandbytes transformers sentence-transformers faiss-gpu
!pip -q install langchain openai tiktoken chromadb
!pip -q install langchain
!pip install pypdf
!pip install sentence-transformers==2.2.2
!pip install tiktoken
!pip install faiss-cpu

In [ ]:
import pandas as pd
import os
import re

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

In [ ]:
loader = CSVLoader(file_path='/content/final_sundaram_finance_data.csv', source_column="Launch_URL")
data = loader.load()

In [ ]:
data

[Document(page_content="SEGMENT_NAME: Diesel Finance\nPRACTICE_TITLE1: Out of the way servicing to Diesel Finance Customers is Delhi Fleet branch's success formula! in Diesel Finance\n\n\nFocussed on major clients\nowning 200-300 vehicles and who had credit facility with local pumps\n\nHow they did it?\n\n• Met fleet operators\nalong with his preferred oil company executive and our product manager\n\n• They were comfortable\nwith the local pumps who offer credit facility. Explained the benefits of\nswitching to diesel finance. \n\n• Interest was their\nbiggest concern. Reward mechanism was presented in detail and how rewards can\noffset the interest accrued was detailed\n\n• An excel calculator\nwhich calculates the interest and reward was presented to the operators\n\n• Branch made sure to\nsend the consumption report to all big customers every morning\n\n• Again in the evening\nmessage was sent to customer’s portal/account manager on consumption data and\ncut off limit\n\n• Filling o

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(data)

In [ ]:
texts

[Document(page_content="SEGMENT_NAME: Diesel Finance\nPRACTICE_TITLE1: Out of the way servicing to Diesel Finance Customers is Delhi Fleet branch's success formula! in Diesel Finance\n\n\nFocussed on major clients\nowning 200-300 vehicles and who had credit facility with local pumps\n\nHow they did it?\n\n• Met fleet operators\nalong with his preferred oil company executive and our product manager\n\n• They were comfortable\nwith the local pumps who offer credit facility. Explained the benefits of\nswitching to diesel finance.", metadata={'source': 'https://www.sundaramfinance.in/', 'row': 0}),
 Document(page_content='• Interest was their\nbiggest concern. Reward mechanism was presented in detail and how rewards can\noffset the interest accrued was detailed\n\n• An excel calculator\nwhich calculates the interest and reward was presented to the operators\n\n• Branch made sure to\nsend the consumption report to all big customers every morning\n\n• Again in the evening\nmessage was sent t

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
torch.random.manual_seed(0)
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
persist_directory = 'db'
vectordb = Chroma.from_documents(documents=texts,embedding=embeddings,persist_directory=persist_directory)
vectordb.persist()
vectordb = None
vectordb = Chroma(persist_directory=persist_directory,embedding_function=embeddings)

retriever = vectordb.as_retriever(search_kwargs={"k":3})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [ ]:
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# # pip install accelerate
# from transformers import T5Tokenizer, T5ForConditionalGeneration
# model = T5ForConditionalGeneration.from_pretrained(
#     "google/flan-t5-large",
#     device_map="auto",
#     torch_dtype="auto",
#     trust_remote_code=True,
# )
# tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large",trust_remote_code=True)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
template = """As an assistant specializing in question-answering tasks, it is crucial to ensure responses are well-structured and complete. When addressing the query provided, it is imperative to deliver a comprehensive response within a 500-word limit. Please adhere to the following guidelines:

1. If the information necessary to answer the question is available within the documents you have uploaded, please provide a detailed answer. Additionally, cite the source URL from which the answer was derived.
2. If the uploaded documents do not contain the information required to answer the question, clearly state that an answer cannot be provided. Emphasize that this decision is based on the absence of relevant information in the provided documents.
3. It is important to avoid basing your answer on any knowledge external to the content of the uploaded documents. Your responses should be solely informed by the information contained within these documents.
4. In instances where the question pertains to you, please respond with: "I am a SF GenAI Assistant, designed to help you answer questions based on the uploaded documents."
If the question ask about you please response I am a SF GenAI Assistant and help you to answer your question based on the uploaded documents.
Question: {question}
Context: {context}
Answer:
=======
"""

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.3,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=500,
)
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
from langchain.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(template)
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
output_parser = StrOutputParser()
from langchain.chains import LLMChain
llm_chain = LLMChain(llm=llm, prompt=prompt)
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [ ]:
query="What is diesel finance"
result=rag_chain.invoke(query)
print("******"*20)
print("Answer :")
print(result.split("Answer:")[-1].replace("===",'').strip())
print("Source Url:")
# Extract source links using regular expressions
source_links = re.findall(r'source\': \'(.*?)\'', result)
# Print the extracted source links
list_of_source_url=[]
for link in source_links:
  if(link not in list_of_source_url):
    list_of_source_url.append(link)
print(list_of_source_url)

************************************************************************************************************************
Answer :
=
EABOS stands for Employee Assured Bonus Scheme. This scheme has two different programs - one spanning over a period of one year and another extending up to two years. The primary objective of EABOS is to reward and motivate employees who demonstrate loyalty through their long-term service at the company. For more specific queries regarding EABOS or other aspects related to it, it is recommended to consult with either the Human Resources (HR) department or the Faculty Head & Registrar (FHR). These contacts can provide further clarification and guidance as needed.
Source Url:
['https://sfllms.sundaramgurukulam.in/approot/redirecttopage?url=elearn/courses/VTJGc2RHVmtYMStjaG81aFJlbTZuZEFyN1VKYUhIL083VE8vazBMd0N2dz0=/details/VTJGc2RHVmtYMThnU2hDTTR3VE1uODVkVlArRWgxZlp1UnJ4SklhcHVDVT0=', 'https://www.sundaramfinance.in/']


In [ ]:
print(result)

Human: As an assistant specializing in question-answering tasks, it is crucial to ensure responses are well-structured and complete. When addressing the query provided, it is imperative to deliver a comprehensive response within a 500-word limit. Please adhere to the following guidelines:

1. If the information necessary to answer the question is available within the documents you have uploaded, please provide a detailed answer. Additionally, cite the source URL from which the answer was derived.
2. If the uploaded documents do not contain the information required to answer the question, clearly state that an answer cannot be provided. Emphasize that this decision is based on the absence of relevant information in the provided documents.
3. It is important to avoid basing your answer on any knowledge external to the content of the uploaded documents. Your responses should be solely informed by the information contained within these documents.
4. In instances where the question pertains